# Supervised Finetunine of Falcon 7B model on Norewegian Dialogue Dataset

In [1]:
## Inference
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=False,
)
device_map = {"": 0}



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/lars/miniconda3/envs/llm/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/lars/miniconda3/envs/llm/lib/libcudart.so'), PosixPath('/home/lars/miniconda3/envs/llm/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


### Load Guanaco Instruction Dataset


In [2]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

Found cached dataset json (/home/lars/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-c93588435bc90172/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


### Custom dataset - Norwegian Dialogue Generation Disclaimer this dataset was generated using GPT4 and should only be used for non commercial purposes.

In [3]:

# Load the transformed data into a HuggingFace Dataset
dataset = load_dataset('json', data_files='transformed.json', split='train')
validation_dataset = load_dataset('json', data_files='transformed_valdiation.json', split='train')
# Let's check the first element in the dataset
print(dataset[0])
print(validation_dataset[0])

Found cached dataset json (/home/lars/.cache/huggingface/datasets/json/default-b55761bd55293d82/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Found cached dataset json (/home/lars/.cache/huggingface/datasets/json/default-764d7df3d9be3714/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


{'id': '0', 'text': '### Human Please generate a dialogue between Erik and Kristin, two Norwegians in their forties discussing an upcoming reunion party. Keep it colloquial. ### Assistant Erik: Hei Kristin, har du hørt om gjenforeningsfesten som skal arrangeres neste måned?\nKristin: Hei Erik! Ja, jeg har fått invitasjonen. Det er jo utrolig at det er 20 år siden vi gikk ut av videregående, ikke sant? Tiden flyr ass!\nErik: Absolutt! Det er helt latterlig hvordan tiden har gått. Tenk at vi alle samles igjen etter så lang tid. Planlegger du å gå da?\nKristin: Ja, jeg har tenkt å dra dit. Det kan være gøy å se alle sammen igjen og høre hva de driver med nå, liksom. Hva med deg?\nErik: Ja, jeg skal også dit. Jeg er spent på å se hvor mye folk har forandret seg og høre historiene deres. Det blir nok noen gode mimrestunder!\nKristin: Hehe, det er sikkert! Men, ærlig talt, jeg er litt nervøs også. Jeg har ikke sett noen av dem på så lenge, og jeg er redd for at det kan bli litt kleint.\nErik

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
## Finetune

from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)



In [7]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"
eval_steps = 10
do_eval = True
evaluation_strategy = "steps"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    eval_steps=eval_steps,
    do_eval=do_eval,
    evaluation_strategy=evaluation_strategy,
)

In [8]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Loading cached processed dataset at /home/lars/.cache/huggingface/datasets/json/default-b55761bd55293d82/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-bcbb27c603de76b4.arrow
Loading cached processed dataset at /home/lars/.cache/huggingface/datasets/json/default-764d7df3d9be3714/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-391ac2f8062796bd.arrow


In [9]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

### Finetune Model - Beware! If your dataset is small it will overfit super quickly!

In [10]:

trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,1.882900,1.776837


KeyboardInterrupt: 


## Load vanilla model and run inference for crude comparison

In [11]:

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_4bit=True
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
%%timeit -r 5
### Inference

# inference test
with torch.inference_mode():
    text = '#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.to(model.device), 
                            max_new_tokens=30, 
                            temperature=0.5, 
                            num_return_sequences=1,
                            do_sample=True,
                            use_cache = True)

print(tokenizer.batch_decode(outputs))


/home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Hmmmm, det er jo ikke alltid så bra når man ikke har jobb. Men det er jo ikke så lett å']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Ja, det er jo ikke alltid lett å finne en jobb. Hvordan går det med dine studier?\n\nW']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Hm, det er jo veldig vanskelig. Har du ikke lest meg?\n\nWaleed: Ja, jeg']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Ja, jeg har ikke hatt noe jobb i lenge. Jeg har vært på jobbe i Roma og i']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Ja, det er jo ikke så lett i dag. Men hva skal man gjøre?\n\nWaleed: Jeg']
['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbinte


## Load finetuned model and run inference for crude comparison

In [14]:

# Load finetuned model 

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_4bit=True
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
# change this path to your checkpoint folder path
model_id = "/home/lars/ML/LLM/results/checkpoint-30"
model = PeftModel.from_pretrained(model, model_id)
model.config.use_cache = False




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
%%timeit -r 5
### Inference

# inference test
with torch.inference_mode():
    text = '#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.to(model.device), 
                            max_new_tokens=30, 
                            temperature=0.5, 
                            num_return_sequences=1,
                            do_sample=True,
                            use_cache = True)

print(tokenizer.batch_decode(outputs))



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Åh, det høres ikke bra ut. Har du prøvd å jobbe med sosiale medier eller noen annen']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Åh, det er så vanskelig med jobbmarkeder nå. Men du er så flink, jeg er sikker på']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Åh, det er så vanskelig å høre. Har du lagt noen annonser eller gjort en søkn']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Åh, det er så vanskelig. Hva med å finne en god jobb? Vi kan være en team og hj']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana: Åh, det er skjønt. Men du skal nok finne noe. Hva med å ta en kaffe s']
['#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervj